In [3]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Float, Thread
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap
from mitsuba.core import Color3f

# Load the Cornell Box
Thread.thread().file_resolver().append('C:/Users/Max/Code/mitsuba2/resources/data/scenes/bunny/')
scene = load_file('C:/Users/Max/Code/mitsuba2/resources/data/scenes/bunny/bunny.xml')

# Find differentiable scene parameters
params = traverse(scene)

print(params)

2021-09-07 21:25:23 INFO main [xml.cpp:1221] Loading XML file "C:\Users\Max\Code\mitsuba2\resources\data\scenes\bunny\bunny.xml" ..

2021-09-07 21:25:23 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-09-07 21:25:23 INFO main [xml.cpp:355] "C:\Users\Max\Code\mitsuba2\resources\data\scenes\bunny\bunny.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-09-07 21:25:23 WARN main [PLYMesh] "bunny.ply": performance warning -- this file uses the ASCII PLY format, which is slow to parse. Consider converting it to the binary PLY format.

2021-09-07 21:25:23 WARN main [PLYMesh] Attribute without postifx are not handled for now: attribute "confidence" ignored.

2021-09-07 21:25:23 WARN main [PLYMesh] Attribute without postifx are not handled for now: attribute "intensity" ignored.

2021-09-07 21:25:23 INFO main [Scene] Building scene in OptiX ..

ParameterMap[
    my_envmap.scale,
  * my_envmap.data,
    my_envmap.resolution,
    PLYMesh.to_world,
  * PLYMesh.bsdf.specular_reflectance.value,
  * PLYMesh.bsdf.eta.value,
  * PLYMesh.bsdf.k.value,
    PLYMesh.vertex_count,
    PLYMesh.face_count,
  * PLYMesh.faces_buf,
  * PLYMesh.vertex_positions_buf,
  * PLYMesh.vertex_normals_buf,
  * PLYMesh.vertex_texcoords_buf,
    PerspectiveCamera.near_clip,
    PerspectiveCamera.far_clip,
    PerspectiveCamera.focus_distance,
    PerspectiveCamera.shutter_open,
    PerspectiveCamera.shutter_open_time,
]


In [4]:
# Make a backup copy
param_res = params['my_envmap.resolution']
param_ref = Float(params['my_envmap.data'])

# Discard all parameters except for one we want to differentiate
params.keep(['my_envmap.data'])

# Render a reference image (no derivatives used yet)
image_ref = render(scene, spp=16)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_ref.png', image_ref, crop_size)

![out_ref.png](out_ref.png)

In [7]:
# Change to a uniform white lighting environment
params['my_envmap.data'] = ek.full(Float, 1.0, len(param_ref))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=.02)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_start.png', image, crop_size)
write_bitmap('temp/envmap_start.png', params['my_envmap.data'],
                 (param_res[0], param_res[1]))

for it in range(100):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(param_ref - params['my_envmap.data']))
    print('Iteration %03i: error=%g' % (it, err_ref[0]))


image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_end.png', image, crop_size)
write_bitmap('temp/envmap_end.png', params['my_envmap.data'],
                 (param_res[0], param_res[1]))


Iteration 000: error=85794.4
Iteration 001: error=83035.1
Iteration 002: error=80315.2
Iteration 003: error=77646.2
Iteration 004: error=75030.6
Iteration 005: error=72471.5
Iteration 006: error=69970.6
Iteration 007: error=67528
Iteration 008: error=65145.6
Iteration 009: error=62824
Iteration 010: error=60563.8
Iteration 011: error=58365.8
Iteration 012: error=56230.7
Iteration 013: error=54157.6
Iteration 014: error=52147.2
Iteration 015: error=50199
Iteration 016: error=48312.4
Iteration 017: error=46487.8
Iteration 018: error=44725.2
Iteration 019: error=43023.4
Iteration 020: error=41381.9
Iteration 021: error=39800.3
Iteration 022: error=38278.2
Iteration 023: error=36813.8
Iteration 024: error=35406.4
Iteration 025: error=34055.1
Iteration 026: error=32759.2
Iteration 027: error=31517.4
Iteration 028: error=30328.1
Iteration 029: error=29191
Iteration 030: error=28104.8
Iteration 031: error=27067.8
Iteration 032: error=26078.3
Iteration 033: error=25135.1
Iteration 034: error=2

![start](temp/out_start.png)
![end](temp/out_end.png)
![env_start](temp/envmap_start.png)
![env_end](temp/envmap_end.png)